# Data Wrangling: Join, Combine, and Reshape Data 

In [1]:
# Much of programming work in data analysis and modeling is spent on data preparation:
# Loading, cleaning, transforming, and rearranging.
# Sometimes the way that data is stored in files or databases is not the way you need it for a data processing application. 
# Many people choose to do and hoc processing of data from one form to another using a general purpose programming language, 
# like Python, Perl, R, or Java, or Unix text processing tools like sed or awk. 


# Combining and Merging Data Sets

In [2]:
# Data contained in pandas object can be combined together in a number of ways. 
# The three main ways of combining data are merges, joins, and concatenation.
# This section focuses on the mechanics of joining data sets with pandas.

# Database-style DataFrame merges

In [3]:
# Merge or join operations combine data sets by linking rows using one or more keys. 
# These operations are central to relational databases. 
# The merge function in pandas is the main entry point for using these algorithms on your data. 
# Let's start with a simple example: 
import pandas as pd 
import numpy as np
from pandas import Series, DataFrame

df1 = DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],'data1': range(7)}) 
df2 = DataFrame({'key': ['a', 'b', 'd'],'data2': range(3)}) 

In [4]:
df1 

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [5]:
df2

,key,data2
0,a,0
1,b,1
2,d,2


In [6]:
# This is an example of a many-to-one merge situation;
# the data in df1 has multiple rows labeled a and b, whereas df2 has only one row for each value in the key column. 
# Calling merge with these objects we obtain:
pd.merge(df1, df2) 

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [7]:
# Note that I didn't specify which column to join on.
# If that information is not specified, merge uses the overlapping column names as the keys. 
# It's good practice to specify explicitly, though:
pd.merge(df1,df2, on='key') 

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [8]:
# If the column names are different in each object, you can specify them separately: 
df3 = DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],'data1': range(7)}) 
df4 = DataFrame({'rkey': ['a', 'b', 'd'],'data2': range(3)})
pd.merge(df3, df4, left_on='lkey', right_on='rkey') 

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


In [9]:
# You may notice that the 'c' and 'd' values and associated data are missing from the result.
# By default merge does an 'inner' join; the keys in the result are the intersection, or the common set found in both tables.
# Other possible options are 'left', 'right', and 'outer'.
# The outer join takes the union of the keys, combining the effect of applying both left and right joins:
pd.merge(df1, df2, how='outer') 

,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


In [10]:
# Many-to-many merges have well-defined though not necessarily intuitive behavior. 
# Here's an example: 
df1 = DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],'data1': range(6)}) 
df2 = DataFrame({'key': ['a', 'b', 'a', 'b', 'd'],'data2': range(5)}) 

In [11]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [12]:
df2 

,key,data2
0,a,0
1,b,1
2,a,2
3,b,3
4,d,4


In [13]:
pd.merge(df1, df2, on='key', how='left') 

,key,data1,data2
0,b,0,1.0
1,b,0,3.0
2,b,1,1.0
3,b,1,3.0
4,a,2,0.0
5,a,2,2.0
6,c,3,NaN
7,a,4,0.0
8,a,4,2.0
9,b,5,1.0


In [14]:
# Many-to-many joins form the Cartesian product of the rows. 
# Since there were three 'b' rows in the left DataFrame and two in the right one, 
# there are six 'b' rows in the result. 
# The join method only affects the distinct key values appearing in the result:
pd.merge(df1, df2, how='inner') 

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,b,5,1
5,b,5,3
6,a,2,0
7,a,2,2
8,a,4,0
9,a,4,2


In [15]:
# To merge with multiple keys, pass a list of column names: 
left = DataFrame({
    'key1': ['foo', 'foo', 'bar'],
    'key2': ['one', 'two', 'one'],
    'lval': [1, 2, 3]
    }) 
right = DataFrame({
    'key1': ['foo', 'foo', 'bar', 'bar'],
    'key2': ['one', 'one', 'one', 'two'],
    'rval': [4, 5, 6, 7]
    }) 

In [16]:
pd.merge(left, right, on=['key1', 'key2'], how='outer')

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


In [17]:
# To determine which key combinations will appear in the result depending on the choice of merge method, 
# think of the multiple keys as forming an array of tuples to be used as a single join key 
# (even though it's not actually implemented that way). 
# The default merge method, 'inner', will only result in key combinations observed in both tables:
pd.merge(left, right, on='key1') 

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [18]:
# A last issue to consider in merge operations is the treatment of overlapping column names. 
# While you can address the overlap manually (see the earlier section on renaming axis labels),
# merge has a suffixes option for specifying strings to append to overlapping names in the left and right DataFrame objects:
pd.merge(left, right, on='key1', suffixes=('_left', '_right')) 

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [19]:
# Merge Function arguments 
# Argument Description 
# left >>>>>> DataFrame to be merged on the left side 
# right >>>>> DataFrame to be merged on the right side
# how >>>>>> One of 'inner', 'outer', 'left', or 'right'; default 'inner'
# on >>>>>>>>  Column names to join on. Must be found in both DataFrame objects. If not specified and no other join keys given, 
# will use the intersection of the column names in left and right as the join keys
# left_on >>>>> Columns in left DataFrame to use as join keys
# right_on >>>>> Analogous to left_on for left DataFrame
# left_index >>>> Use row index in left as its join key (or keys, if a MultiIndex)
# right_index >>>>>> Analogous to left_index
# sort >>>>>>> Sort merged data lexicographically by join keys; 
# True by default (disable to get better performance in some cases on large datasets)
# suffixes >>>>>> Tuple of string values to append to column names in case of overlap; default ('_x', '_y') 
# copy >>>>>>> If False, avoid copying data into resulting data structure in some exceptional cases; default True 

# Merging on Index

In [20]:
# In some cases, the merge key or keys in a DataFrame will be found in its index.
# In this case, you can pass left_index=True or right_index=True (or both) to indicate that the index should be used as the merge key: 
left1 = DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'],'value': range(6)}) 
right1 = DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])


In [21]:
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [22]:
right1 

,group_val
a,3.5
b,7.0


In [23]:
pd.merge(left1, right1, left_on='key', right_index=True) 

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


In [24]:
# Since the default merge method is to intersect the join keys,
# you can instead form the union of them with an outer join:
pd.merge(left1, right1, left_on='key', right_index=True, how='outer') 

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


In [25]:
# With hierarchically-indexed data, things are more complicated,
# as joining on index is implicitly a multiple-key merge:
lefth = DataFrame({
    'key1': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada'],
    'key2': [2000, 2001, 2002, 2001, 2002],
    'data': np.arange(5.)
    }) 

In [26]:
right = DataFrame(np.arange(12).reshape((6, 2)),
    index=[['Nevada', 'Nevada', 'Ohio', 'Ohio', 'Ohio', 'Ohio'],
    [2001, 2000, 2000, 2000, 2001, 2002]],
    columns=['event1', 'event2'])


In [27]:
lefth

,key1,key2,data
0,Ohio,2000,0.0
1,Ohio,2001,1.0
2,Ohio,2002,2.0
3,Nevada,2001,3.0
4,Nevada,2002,4.0


In [28]:
right

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In [31]:
# In this case you have to indicate multiple columns to merge on as a list
# (note the handling of duplicate index values with how='outer') 
pd.merge(lefth, right, left_on=['key1', 'key2'], right_index=True)

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4,5
0,Ohio,2000,0.0,6,7
1,Ohio,2001,1.0,8,9
2,Ohio,2002,2.0,10,11
3,Nevada,2001,3.0,0,1


In [30]:
pd.merge(lefth, right, left_on=['key1', 'key2'], right_index=True, how='outer') 

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4.0,5.0
0,Ohio,2000,0.0,6.0,7.0
1,Ohio,2001,1.0,8.0,9.0
2,Ohio,2002,2.0,10.0,11.0
3,Nevada,2001,3.0,0.0,1.0
4,Nevada,2002,4.0,NaN,NaN
4,Nevada,2000,NaN,2.0,3.0


In [32]:
# Using the indexes of both sides of the merge is also not an issue:
left2 = DataFrame([[1., 2.], [3., 4.], [5., 6.]], index=['a', 'c', 'e'],columns=['Ohio', 'Nevada'])

In [33]:
right2 = DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],index=['b', 'c', 'd', 'e'],columns=['Missouri', 'Alabama']) 


In [34]:
left2

,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


In [35]:
right2

,Missouri,Alabama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


In [36]:
pd.merge(left2, right2, how='outer', left_index=True, right_index=True) 

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [37]:
# DataFrame has a more convenient join instance for merging by index. 
# It can also be used to combine together many DataFrame objects having the same or similar 
# indexes but non-overlapping columns. 
# In the prior example, we could have written: 
left2.join(right2, how='outer') 

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [39]:
# In part for legacy reasons (i.e., much earlier versions of pandas), 
# DataFrame's join method performs a left join on the join keys, exactly preserving the left frame'set row index. 
# It also supports joining the index of the passed DataFrame on one of the columns of the calling DataFrame: 
left1.join(right1, on='key') 

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


In [40]:
# Lastly, for simple index-on-index merges, you can pass a list of DataFrames to join as an alternative to using 
# the more general concat function described in the next section:
another = DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
                    index=['a', 'c', 'e', 'f'],
                    columns=['New York', 'Oregon'])

In [41]:
left2.join([right2, another]) 

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0


In [42]:
left2.join([right2, another], how='outer') 

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0
b,NaN,NaN,7.0,8.0,NaN,NaN
d,NaN,NaN,11.0,12.0,NaN,NaN
f,NaN,NaN,NaN,NaN,16.0,17.0


# Concatenating Along an Axis 

In [43]:
# Another kind of data combination operation is alternatively referred to as concatenation, binding, or stacking. 
# Numpy's has concatenate function for doing this with raw NumPy arrays:
arr = np.arange(12).reshape((3,4)) 
arr 

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [44]:
np.concatenate([arr, arr], axis=1)  

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

In [45]:
# In the context of pandas objects such as Series and DataFrame,having labeled axes enable you to further generalize array concatenation. 
# In particular, you have a number of additional things to think about:

# If the objects are indexed differently on the other axes, 
# should we combine the distinct elements in these axes or use only the shared values (the intersection)?
# Do the concatenated chunks of data need to be identifiable in the resulting object? 
# Does the concatenation axis matter at all? 
# In the simplest case, concatenating Series along a particular axis: 
s1 = Series([0, 1], index=['a', 'b']) 
s2 = Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = Series([5, 6], index=['f', 'g'])


In [49]:
# Calling concat with these objects in a list glues together the values and indexes: 
pd.concat([s1, s2, s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

In [50]:
# By default concat works along axis=0, producing another Series.
# If you pass axis=1, the result will instead be a DataFrame (axis=1 is the columns):
pd.concat([s1, s2, s3], axis=1) 

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [52]:
# In this case there is no overlap on the other axis, which you can see is the sorted 
# union (the 'outer' join) of the indexes. You can instead intersect them by passing join='inner': 
s4 = pd.concat([s1 * 5, s3]) 

In [53]:
pd.concat([s1, s4], axis=1)

,0,1
a,0.0,0
b,1.0,5
f,NaN,5
g,NaN,6


In [54]:
pd.concat([s1, s4], axis=1, join='inner') 

,0,1
a,0,0
b,1,5
